# Module Import

In [86]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

import os



# Model Client Initialization

In [87]:
ollama_host = "http://host.docker.internal:11434"
ollama_model = 'nexusraven:latest'


# Context

In [88]:
context = {
    'patients' : {
        'A00001' : {
            'dni':'74324694A',
            'name':'Fulgoroncio',
            'age':35,
            'clinical_records': [
                {
                    'record_id':'V00001',
                    'date':'2024-11-15',
                    #'patient_id': 'A00001',
                    'disease_id': 'X00005',
                    'treatment': 'Buy a hammer and carry everywhere to break the copy reference.',
                    'status':'Healed',
                    'observation':'She may need another solution in prision.'
                }
            ]
        },
        'A00002' : {
            'dni':'24336634A',
            'name':'Petunia',
            'age':27,
            'clinical_records':[
                {
                    'record_id':'V00001',
                    'date':'2024-11-15',
                    #'patient_id': 'A00002',
                    'disease_id': 'X00005',
                    'treatment': 'Buy a hammer and carry everywhere to break the copy reference.',
                    'status':'Healed',
                    'observation':'She may need another solution in prision.'
                }
            ]
        },
    },
    'diseases' : {
        'X00001' : {
            'disease_name' : 'Invisible toy disorder',
            'description': 'The person tends to loose members of toys. Also suffers from cognitive time jumps, as she forgots 10 minute lapses.'
        },
        'X00002' : {
            'disease_name' : 'Sofacosis syndrome',
            'description': 'Mental syndrome to get sofas placed in an exact location on images.'
        },
        'X00003' : {
            'disease_name' : 'Spontaneous mattress somnambulism',
            'description': 'The person awakes in another room, with a matrees moved from a room to another. In some cases the person develops a phobia to hear possible solutions.'
        },
        'X00004' : {
            'disease_name' : 'Indian immune repulsive',
            'description': 'The person naturally causes indian people to vanish or avoid responses to her inqueries. on its chronicle phase it generates unpleasent behaviors from greeks'
        },
        'X00005' : {
            'disease_name' : 'Mirror Mimic Madness',
            'description': 'The person believes they are a mirror and must copy the exact movements of whoever is in front of them—even strangers in public.'
        },
        'X00006' : {
            'disease_name' : 'Dramatic Slow-Mo Virus',
            'symptoms': 'The infected person perceives their life as a dramatic movie and moves in slow motion during mundane tasks.'
        }
    }
    
}

# Tools


## Pydantic Data Classes

Pydantic classes are used only to create the definition for the arguments.

In this case we also add tags ['...'] to descriptions to give more relevant information to the model

In [89]:
class hospital_patients(BaseModel):
    pass
    
class patient_clinical_record(BaseModel):
    patient_id: str = Field(description='[patiend-identifier] System identifier of the patient.')
    

## Functions

The tool tag is used to include the class descriptions to the functions. This option allows to create the scheme in a more organized form.

In [90]:
@tool(args_schema = hospital_patients)
def get_hospital_patients() -> dict:
    """
    [all-patients] This function retrieves the patients registered in the hospital.

    Returns: A dictionary of the patients registered in the hospital.
    """
    
    patients = {
        key: { attr:attr_data for attr,attr_data in data.items() if attr != 'clinical_records' }
        for key,data in context['patients'].items()
    }

    return {'patients': patients}

@tool(args_schema = patient_clinical_record)
def get_patient_clinical_record(patient_id:str) -> dict:
    '''
    [clinical-info] Retrieves the clinical history of a patient by the sistem identifier, which has a numerical text portion.
    
    Returns: Dictionary with the clinical record of the requested patient.
    
    Args description:
        patient_id (str): [patiend-identifier] System identifier of the patient.
    '''
    
    if  patient_id in context['patients'].keys():
        records = context['patients'].get(patient_id)['clinical_records']
        return records
    else:
        print("No patient found with id {}".format(patient_id))
        return None
    

## Tool Schema Formatting

In [91]:
tools = [
    get_hospital_patients,
    get_patient_clinical_record
]

function_map = {tool.name: tool for tool in tools}
function_map


{'get_hospital_patients': StructuredTool(name='get_hospital_patients', description='[all-patients] This function retrieves the patients registered in the hospital.\n\nReturns: A dictionary of the patients registered in the hospital.', args_schema=<class '__main__.hospital_patients'>, func=<function get_hospital_patients at 0x72e0fd55fd80>),
 'get_patient_clinical_record': StructuredTool(name='get_patient_clinical_record', description='[clinical-info] Retrieves the clinical history of a patient by the sistem identifier, which has a numerical text portion.\n\nReturns: Dictionary with the clinical record of the requested patient.\n\nArgs description:\n    patient_id (str): [patiend-identifier] System identifier of the patient.', args_schema=<class '__main__.patient_clinical_record'>, func=<function get_patient_clinical_record at 0x72e0fd55fc40>)}

# Agent Call

In [92]:

from langchain_ollama import ChatOllama
from langchain.schema.runnable import RunnableLambda

import re

class mychat():
    def __init__(self,tools):
        #Tools
        self.function_map = {tool.name:tool for tool in tools}
        self.tools_schema = [convert_to_openai_function(tool) for tool in tools]
        self.tools_prompt = self.build_tools_prompt()

        #Prompt
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.tools_prompt),
            MessagesPlaceholder(variable_name='chat_history'),
            ('user','{user_query}')
        ])
        
        #Model setup        
        self.model = ChatOllama(
            model="nexusraven:latest",
            base_url=ollama_host,
            temperature=0.001
        )
        
        self.chat_history = []
        self.intermedium_steps = []
        self.chain = self.prompt | self.model | RunnableLambda(self.parse_functions)
    
    
        
    def conversation_chain(self,user_query):
        print('Entered on function')
        response = self.chain.invoke({
            "chat_history": self.chat_history,
            "user_query": user_query
        })

        self.chat_history.append(HumanMessage(content=user_query))
        self.chat_history.extend(self.intermedium_steps)
        self.chat_history.append(AIMessage(content="\n".join(map(str, response['result']))))
        self.intermedium_steps = []
        
        print(self.chat_history)
        
        return response['result']
    
    
    def clean_history(self):
        self.chat_history = []
        self.intermedium_steps = []
        
    def build_tools_prompt(self):   
        function_prompt = ''
        for funct in self.function_map.values():  
            function_prompt += ( 'def {tool_name}({args}):\n"""\n{desc}\n"""\n\n'.format(
                tool_name = funct.name, 
                args = ','.join(['{}:{}'.format(key,data['type']) for key, data in funct.args.items() ]),
                #args = inspect.signature(funct),
                desc = funct.description
            ))
            
        instruction_prompt = '''
    Instructions:
    - ONLY use the provided functions.
    - DO NOT nest function calls.
    - Use ONLY values that have already been returned in previous steps.
    - Use the exact function name and parameters as defined.
    - If no parameters are needed, call without arguments
    - Output format must be exactly: Call: function_name(arguments)<bot_end>
        '''
        
        system_prompt = function_prompt + instruction_prompt
        
        print('Returning prompt')
        
        return system_prompt
    
    def parse_functions(self, response):
        
        print('Ready to parse the response')
        response_message = response.content
        self.intermedium_steps.append(AIMessage(content=response.content))
        
        #We look for function calls located between the strings "Call:" and "<bot_end>"
        matches = re.findall(r'Call:\s*(\w+)\((.*?)\)<bot_end>', response_message)

        print(matches)
        #We assure we got function calls in the response
        results = []
        if matches:
            #This loop runs througth the sequence of functions included in the response
            #for func in matches:
            for func_name,args_text in matches:
                try:
                    #This executes our functions
                    print(args_text)
                    args = eval(f'dict({args_text})') if (args_text.strip()) and (args_text != 'null=null') else {}
                    self.intermedium_steps.append(AIMessage(content='Called function: {}({})'.format(func_name,args_text)))
                    
                    result = self.function_map[func_name].run(args)
                    results.append(result)
                except Exception as e:
                    result = "Function call failed: {}".format(func_name)
                    results.append(result)
        else:
            results.append("No function calls found.")
        return {'result':results}
        

In [74]:
chat = mychat(tools)
resp = chat.conversation_chain('can you get the patients of the hospital?')

#can you get the patients of the hospital?
#show me the clinical record of fulgoroncio


Returning prompt
Entered on function
Ready to parse the response
[('get_hospital_patients', 'null=null')]
null=null
[HumanMessage(content='can you get the patients of the hospital?', additional_kwargs={}, response_metadata={}), AIMessage(content=' \nCall: get_hospital_patients(null=null)<bot_end> \nThought: The function call `get_hospital_patients(null=null)` answers the question "can you get the patients of the hospital?" because it retrieves the patients registered in the hospital.\n\nThe `get_hospital_patients` function is defined to return a dictionary of the patients registered in the hospital, and the `null` parameter is used to indicate that no specific patient is being requested. Therefore, by calling `get_hospital_patients(null=null)`, we are asking for all the patients registered in the hospital.\n\nThe function call does not nest any other function calls, so it does not rely on any other functions or values returned by previous steps. It only uses the provided functions and 

In [ ]:
resp = chat.conversation_chain('show me the clinical record of fulgoroncio?')